# Sbtee!

Analyzing sbt errors

# How does it work?

```bash

alias sbt='f() { sbt "$@" |tee -a ~/.sbtlogs; };f'

```

# Data Exploration

Aug 31, 2016 to Dec 23, 2016 (~4 months)


In [77]:
cat sbt_usage|wc -l

 3939874


In [78]:
ls -lh|grep sbt_usage

-rw-r--r--  1 cale  staff    48M Dec 24 00:18 sbt_usage


# Sbt logs

In [79]:
cat sbt_usage|grep '0minfo'|wc -l

  119223


In [80]:
cat sbt_usage|grep '32msuccess'|wc -l

    2101


In [81]:
cat sbt_usage|grep '33mwarn'|wc -l

   68851


In [82]:
cat sbt_usage|grep '31merror'|wc -l

   97093


**Note**: '31m' and '33m' are color escape sequences

# Compilation Errors

In [107]:
with open("sbt_usage", "r") as f:
    sbt_usage = f.readlines()
    usage_pairs = list(zip(sbt_usage[:-1], sbt_usage[1:]))
    compilations = [b for (a, b) in usage_pairs if '0mCompiling' in a]
    errors = [c for c in compilations if '31merror' in c]
    (errors_n, compilations_n) = len(errors), len(compilations)
    errors_percentage = errors_n/compilations_n*100
    print("Compilations: %(compilations_n)i" % locals())
    print("Errors: %(errors_n)i" % locals())
    print("Percentage: %(errors_percentage)i%%" % locals())

Compilations: 5988
Errors: 3361
Percentage: 56%


# Top 10 errors

In [2]:
import re; from collections import Counter, OrderedDict

def truncate(line):
    p = re.compile('.scala:\d+:') 
    m = p.search(line)
    if m is None: end = len(line)
    else: end = m.end()
    return line[end:]

with open("sbt_errors", "r") as f: #cat sbt_usage|grep '31merror'
    rawErrors = [truncate(l) for l in f.readlines()]
    groupedErrors = Counter(rawErrors)
    mostCommonErrors = groupedErrors.most_common(6)[1:]
    [print(e) for e in mostCommonErrors]

(' type mismatch;\x1b\n', 1771)
(' not found: value Get\x1b\n', 136)
(' too many arguments for method parameters: (pdm: spray.routing.directives.ParamDefMagnet)pdm.Out\x1b\n', 128)
(' not found: value UserRole\x1b\n', 119)
(' not found: value check\x1b\n', 107)


# Errors per type

In [1]:
with open("sbt_errors", "r") as f: #cat sbt_usage|grep '31merror'
    errors = f.readlines()
    not_found_n = len([e for e in errors if 'not found: value ' in e])
    too_many_arguments_n = len([e for e in errors if 'too many arguments for method parameters: ' in e])
    type_mismatch_n = len([e for e in errors if 'type mismatch;' in e])
    takes_type_parameters_n = len([e for e in errors if 'takes type parameters' in e])
    polymorphic_expressions_n = len([e for e in errors if 'polymorphic expression cannot be instantiated to expected type' in e])
    exploding_macros_n = len([e for e in errors if 'exception during macro expansion' in e])
    
    print("Not found: %(not_found_n)i" % locals())
    print("Too many arguments: %(too_many_arguments_n)i" % locals())    
    print("Type mismatch: %(type_mismatch_n)i" % locals())        
    print("Takes type parameters: %(takes_type_parameters_n)i" % locals())
    print("Polymorphic expressions: %(polymorphic_expressions_n)i" % locals())
    print("Exploding macros: %(exploding_macros_n)i" % locals())

Not found: 3175
Too many arguments: 128
Type mismatch: 1771
Takes type parameters: 192
Polymorphic expressions: 83
Exploding macros: 61


# Conclusions
- Compiling code is useful
    - 1771 Type mismatch (20+ times a day)
- Most of my errors are due to bad imports
    - 3175 Not found errors (~40 times a day)
- I should probably revise the magnet pattern
    - 128 too many arguments ... ParamDefMagnet